# 네이버 플레이스 블로그 리뷰 크롤링
#### 2020-05-10 (일) 이한울
---
- 2020-05-10 (일): 플레이스 블로그 리뷰 긁어오기 및 스레딩 적용 완료
---
#### 2020-05-11 (월) 이한울
---
- 스레딩 실행 창에서 driver 경로를 본인의 크롬 드라이버가 있는 경로로 설정 후 실행
- time.sleep을 넉넉하게 주어야 HTTP 에러를 최대한 회피할 수 있습니다.
---
#### 2020-05-14 (목) 이한울
---
- 구글 드라이브 > 팀프로젝트 폴더 > [00]...마리톡 > [03] 데이터 > [03] 네이버 플레이스 리뷰 크롤링 > **naver_place_count_total_2020-05-01_final.csv** 파일을 다운로드, 원하는 경로로 이동시킨 뒤 진행해야 합니다.

In [43]:
import  urllib.request
import selenium
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

from pandas import Series, DataFrame
import pandas as pd
import numpy as np

from datetime import datetime
import time
import sys
import math
import re
from tqdm import tqdm, tqdm_notebook

import selenium.common.exceptions as sce
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

from queue import Queue
from multiprocessing.pool import ThreadPool
from threading import Thread
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

import glob
import fnmatch

In [25]:
# 블로그 검색 결과를 저장할 DataFrame 생성
df_blog_review = \
DataFrame(columns=['store_id', 'store_name', 'title', 'writer', 'link', 'summary', 'blog_name', 'original_index'])

In [26]:
df_blog_error = DataFrame(columns=['store_id','store_name','error_code', 'original_index'])

In [27]:
df_blog_empty = DataFrame(columns=['store_id','store_name','original_index','blog_tab'])

In [65]:
# 블로그 본문을 저장할 빈 DataFrame 생성
df_text = DataFrame(columns=['store_id', 'store_name', 'link', 'main text', 'original_index'])

In [29]:
# 플레이스의 업소별 ID를 담은 csv파일 DataFrame으로 변환
# 본인이 naver_place_count_total_2020-05-01_final.csv를 저장한 경로로 설정
df_place = \
DataFrame(pd.read_csv('./naver-place_csv/naver_place_count_total_2020-05-01_final.csv', sep=',', header=0, index_col=0))
df_place

,store_id,store_id_only_num,store_name,booking,receipt,blog,total
0,s1518997092,1518997092,준오헤어 노원사거리점,2015.0,3.0,15.0,2033.0
1,s1571970153,1571970153,차홍룸 홍대점,2009.0,1.0,10.0,2020.0
2,s1142939643,1142939643,차홍룸 용산센트럴점,1871.0,3.0,11.0,1885.0
3,s1481443590,1481443590,차홍룸 용산점,1541.0,2.0,11.0,1554.0
4,s1177346002,1177346002,차홍룸 양재점,1373.0,1.0,10.0,1384.0
...,...,...,...,...,...,...,...
2722,s38257322,38257322,바바헤어 대치점,1.0,1.0,2.0,4.0
2723,s1069634711,1069634711,정환헤어샵,1.0,1.0,2.0,4.0
2724,s20852762,20852762,머리짱헤어샵,1.0,2.0,1.0,4.0
2725,s36091089,36091089,살롱160,1.0,1.0,2.0,4.0


# 네이버 플레이스 모바일 페이지 크롤링 함수
#### 2020-05-12 (화) 이한울
---
- 스레딩 테스트 중
---
#### 2020-05-14 (목) 이한울
---
- 스레딩 테스트 완료
---

In [30]:
# status가 400, 500인 경우 크롤링하지 못한 업소의 정보를 따로 저장하는 함수
def df_error_save(store_id, store_name, error_code):
    
    global df_blog_error
    global df_place
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    
    df_blog_error = df_blog_error.append({
        'store_id' : store_id
        , 'store_name' : store_name
        , 'error_code' : error_code
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_blog_error

In [31]:
# 크롤링이 완료된 업소의 정보들을 DataFrame에 저장하는 함수
def df_info_save(store_id, store_name, title, blog_writer, link, summary, blog_name):
    
    global df_blog_review
    global df_place
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    
    df_blog_review = df_blog_review.append({
        'store_id' : store_id
        , 'store_name' : store_name
        , 'title' : title
        , 'writer' : blog_writer
        , 'link' : link
        , 'summary' : summary
        , 'blog_name' : blog_name
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_blog_review

In [32]:
# 플레이스 검색 결과 블로그 리뷰가 하나도 존재하지 않는 업소 정보 저장 함수
def df_no_result(store_id, store_name, blog_tab):
    
    global df_blog_empty
    global df_place
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    
    df_blog_empty = df_blog_empty.append({
        'store_id' : store_id
        , 'store_name' : store_name
        , 'blog_tab' : ('https://m.place.naver.com' + blog_tab)
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_blog_empty

In [33]:
'''
웹드라이버 로딩까지 대기 (잡동사니)
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, blog_x_path))).click()
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, \
"//a[@class='verification_vote-btn vote_fb_btn' and \
@id='verification_vote_btn']//span[contains(.,'Tap To Vote !')]"))).click()
'''

'\n웹드라이버 로딩까지 대기 (잡동사니)\nWebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, blog_x_path))).click()\nWebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a[@class=\'verification_vote-btn vote_fb_btn\' and @id=\'verification_vote_btn\']//span[contains(.,\'Tap To Vote !\')]"))).click()\n'

In [34]:
def m_place_save(driver, store):
    count_list = []    
    
    try:
        store_id = store
        total_number = 1
        store_name = str(df_place[df_place['store_id_only_num'] == store_id]['store_name'].iloc[0])

        driver.get('https://m.store.naver.com/hairshops/detail?entry=plt&id=' + str(store_id)) # df_blog['links'][i]
        status_req = requests.get('https://m.store.naver.com/hairshops/detail?entry=plt&id=' + str(store_id))
        time.sleep(2)
        print(status_req.status_code)
        if status_req.status_code in (list(range(400, 506))):

            print(store, store_name, '크롤링 중 에러가 발생했습니다.')
            print('내부 서버 에러가 발생했습니다.')
            df_error_save(store, store_name, status_req.status_code)
            pass

        elif status_req.status_code == 200:

            time.sleep(1 + (np.random.rand()))
            place_soup = BeautifulSoup(driver.page_source, 'html.parser')
                    
            if place_soup.find_all('span', '_1Y6hi'):            
            
                span = place_soup.find_all('span', '_1Y6hi')[-1]
                link = (span.a).attrs['href']

                if '/review/ugc' in link:

                    print(store, store_name, '크롤링을 진행합니다.\n')
                    blog_tab = link
                    total_count = (((span.a).get_text()).strip()).replace('블로그리뷰 ', '')
                    print(store_name + '의 명시적 총 블로그 리뷰 수:', int(total_count), '\n')
                    print('▶▶', store_name, '의 블로그 탭:', blog_tab, '\n')
                    time.sleep(2)
                    driver.get('https://m.place.naver.com' + blog_tab)
                    req = requests.get('https://m.store.naver.com' + blog_tab)

                    time.sleep(1 + (np.random.rand() + np.random.rand()))

                    if int(total_count) > 10:
                        # 블로그 리뷰 페이지의 다음 버튼 클릭
                        blog_x_path = '//a[@class="_3iTUo"]'        

                        if driver.find_element(By.XPATH, blog_x_path).is_displayed():

                            try:
                                while driver.find_element(By.XPATH, blog_x_path).is_displayed():

                                    next_btn = driver.find_element(By.XPATH, blog_x_path)
                                    next_btn.click()
                                    time.sleep(1.5)

                            except sce.NoSuchElementException:
                                print('해당 엘리먼트를 찾지 못했습니다.')

                            except sce.ElementNotInteractableException:
                                print('해당 엘리먼트를 찾던 중 에러가 발생했습니다.')

                            time.sleep(2)
                            review_soup = BeautifulSoup(driver.page_source, 'html.parser')

                            # 첫 번째 페이지의 블로그 이름, 포스트 제목, 링크, 본문 요약 가져오기  
                            blog_src = ((review_soup.find('div', class_='place_section undefined')).find('ul', class_='_1fYfG undefined')).find_all('li', class_="_3xK_0")

                            # 파싱 후 접근하고자 하는 요소가 존재할 때
                            if blog_src != None:
                                for blog in tqdm_notebook(blog_src):

                                    link = (blog.a).attrs['href']
                                    summary = (blog.find_all('div', class_="_3kKqj"))[1].get_text()
                                    title = blog.find('div', '_2dj8M').get_text()
                                    blog_name = blog.find('span', class_='_1vuEi').get_text()

                                    # 블로그 작성자의 ID를 가져오기 위해 블로그 링크 분할
                                    split_link = re.split('[/+]|[.+]|[?]+', str(link))

                                    # 도메인이 blog.naver.com인 경우 블로그 작성자 가져오기
                                    if split_link[3] == 'blog':
                                        blog_writer = split_link[6]    

                                    else:
                                        pass

                                    if ('cafeRedirect' or 'post.naver.com') not in link:
                                        count_list.append(1)
                                        df_info_save(store_id
                                                     , store_name
                                                     , title
                                                     , blog_writer
                                                     , link
                                                     , summary
                                                     , blog_name)

                            # 파싱 후 접근하고자 하는 요소가 존재하지 않을 때
                            else:
                                print('파싱한 HTML 소스가 존재하지 않습니다!')
                                df_no_result(store, store_name, blog_tab)

                    else:
                        time.sleep(1)
                        review_soup = BeautifulSoup(driver.page_source, 'html.parser')

                        # 첫 번째 페이지의 블로그 이름, 포스트 제목, 링크, 본문 요약 가져오기  
                        blog_src = ((review_soup.find('div', class_='place_section undefined')).find('ul', class_='_1fYfG undefined')).find_all('li', class_="_3xK_0")
                        blog_x_path = None

                        for blog in tqdm_notebook(blog_src):

                            link = (blog.a).attrs['href']
                            summary = (blog.find_all('div', class_="_3kKqj"))[1].get_text()
                            title = blog.find('div', '_2dj8M').get_text()
                            blog_name = blog.find('span', class_='_1vuEi').get_text()

                            # 블로그 작성자의 ID를 가져오기 위해 블로그 링크 분할
                            split_link = re.split('[/+]|[.+]|[?]+', str(link))

                            # 도메인이 blog.naver.com인 경우 블로그 작성자 가져오기
                            if split_link[3] == 'blog':
                                blog_writer = split_link[6]

                            else:
                                pass

                            if ('cafeRedirect' or 'post.naver.com') not in link:
                                count_list.append(1)
                                df_info_save(store_id
                                             , store_name
                                             , title
                                             , blog_writer
                                             , link
                                             , summary
                                             , blog_name)

                        print('다음 페이지가 없습니다.')
                        pass

                else:
                    print('블로그 리뷰가 없습니다.')
                    pass
            
            else:
                print("요청하신 페이지를 찾을 수 없습니다. 검색하신 업소의 정보가 삭제되었습니다.")
                ecode = "요청하신 페이지를 찾을 수 없습니다."
                df_error_save(store, store_name, ecode)
                pass
            
            if len(count_list) == 0:
                print('저장된 블로그 정보가 없습니다.')
                blog_tab = '/hairshop/' + str(store_id) + '/review/ugc'
                df_no_result(store_id, store_name, blog_tab)
            
    except sce.NoSuchElementException:
        print('블로그 리뷰가 없습니다!')
        print('다음 업소의 크롤링을 진행합니다.')
        df_no_result(store, store_name, blog_tab)
        
    except sce.ElementNotInteractableException:
        print('해당 엘리먼트를 찾던 중 에러가 발생했습니다.')
        
    except sce.WebDriverException:
        print('웹 드라이버 에러 발생')
        print('유니코드 변환 에러')
        df_error_save(store, store_name, blog_tab)
    
    print('★', store_name + '의 블로그 리뷰 썸네일 크롤링 성공!')
    driver.close()
    driver.quit()
    
    return df_blog_review

In [35]:
# 크롬 웹 드라이버 헤드리스 설정 및 웹 보안 무시
options = webdriver.ChromeOptions()

prefs = {
    'profile.default_content_setting_values': {
          'cookies': 2, 'images': 2, 'plugins': 2
        , 'popups': 2, 'geolocation': 2, 'notifications': 2
        , 'auto_select_certificate': 2, 'fullscreen': 2, 'mouselock': 2
        , 'mixed_script': 2, 'media_stream': 2, 'media_stream_mic': 2
        , 'media_stream_camera': 2, 'protocol_handlers': 2, 'ppapi_broker': 2
        , 'automatic_downloads': 2, 'midi_sysex': 2, 'push_messaging': 2
        , 'ssl_cert_decisions': 2, 'metro_switch_to_desktop': 2, 'protected_media_identifier': 2
        , 'app_banner': 2, 'site_engagement': 2, 'durable_storage': 2
    }
}

options.add_experimental_option('prefs', prefs)
options.add_argument('--disable-dev-shm-usage')
options.add_argument('User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36')
options.add_argument("Accept=text/html,application/xhtml+xml,application/xml;q=0.9,imgwebp,*/*;q=0.8")
options.add_argument("start-maximized")
options.add_argument('--no-sandbox')
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('headless')
options.add_argument("disable-gpu")
options.add_argument("--disable-web-security")

# 플레이스 블로그 리뷰 썸네일 스레딩
#### 2020-05-14 (목) 이한울
---
- 주석 처리된 부분을 보시면 범위가 나와있습니다.
- a와 end 값을 본인의 범위에 맞게 지정해주신 뒤 실행하시면 됩니다.
- 간혹 지정된 범위까지 크롤링을 하였음에도 계속해서 실행되어 뒷 부분까지 크롤링하는 상황이 발생하니 유의해주시기 바랍니다.
- df_place의 150개 업소 크롤링에 약 43분 소요 [※스레드 3개 기준※]
- df_place의 개 업소 크롤링에 약 분 소요 [※스레드 5개 기준※] 0 ~ 365 16:05분 시작 17:35분 완료
- 0 ~ 365 업소 크롤링 중 에러 발생 없음.
- 업소별 블로그 리뷰 100건 * 스레드 5개 = 5개 업소 크롤링 소요시간: 약 75초 리뷰 수에 따라 가감될 수 있습니다.
- 1인당 약 2시간 정도 소요될 것으로 예상됩니다.

# 아래 셀을 클릭한 뒤 Cell Run All Above 후 아래 셀에서 Ctrl + Enter!

In [13]:
# 플레이스 블로그 크롤링 함수의 스레딩 적용

# =========================================================================
# 총 15743페이지 블로그 리뷰, 약 15만건 이상 블로그 본문 존재
# 1인당 대략 3100페이지의 블로그 리뷰 크롤링

# =========================================================================
# A: start = 0, end = 363 → a: 0      이한울 완료 0 ~ 363 
# df_place['blog'][:321].sum() = 3533
# 2020-05-14 이한울 0 ~ 363 다시 진행중 -> 17시 35분 완료

# A: start = 363, end = 768 → a: 363    2020-05-14 이한울 시작
# df_place['blog'][365:764].sum() = 3326

# B: start = 768, end = 1120 → a: 768

# C: start = 1120, end = 1540 → a: 1120

# D: start = 1540, end = 1885 → a: 1540

# E: start = 1885, end = 2727 → a: 1885

# =========================================================================
# start_no: 본인의 크롤링 시작 index
# csv파일 저장(넘버링)을 위해 본인의 시작 index를 a에 저장
a = 0
# 이한울 original index 366
start_no = 486

# end: 5개씩 스레딩하므로 start_no + 5
end = (start_no + 5)

# end_no: 본인의 크롤링 종료 index. 
end_no = 768

# =========================================================================

# 2020-05-11 (월) 14:46 크롤링 시작

# E를 선택하신 분은 for문의 범위를 따로 잡으셔야 합니다. max_t로 나눴을 때 나머지 없이 딱 맞게 설정해주세요.
max_t = 5

for i in tqdm_notebook(range(int(len(df_place[start_no : end_no]) / max_t))):
    
    q = Queue(maxsize=5)
    for _ in range(q.maxsize):
        driver = Chrome('./chromedriver.exe', options=options)
        q.put(driver)
        
    print(str(start_no) + ' ~ ' + str((end) - 1) + '의 블로그 리뷰 크롤링을 시작합니다.\n')        
    with ThreadPool(5) as tp:
        # 스레드에 입력할 매개변수가 2개 이상일 경우 starmap을 사용한다.
        tp.starmap(
            m_place_save
            , [(q.get(), df_place['store_id_only_num'][ii]) for ii in range(start_no, end)]
        )
        
    start_no += 5
    end += 5
    
    time.sleep(1)
    if start_no > end_no:
        break
        
# 크롤링 후 CSV 파일로 저장하기        
df_blog_review.to_csv('./2020-05-14_place_blog-thumb (%i-%i).csv' %(a, end_no))
df_blog_empty.to_csv('./2020-05-14_place_blog-empty (%i-%i).csv' %(a, end_no))
df_blog_error.to_csv('./2020-05-14_place_blog-error (%i-%i).csv' %(a, end_no))

363 ~ 367의 블로그 리뷰 크롤링을 시작합니다.

1574340058 박호준헤어 일원점 크롤링을 진행합니다.

박호준헤어 일원점의 명시적 총 블로그 리뷰 수: 33 

▶▶ 박호준헤어 일원점 의 블로그 탭: /hairshop/1574340058/review/ugc 

1222327123 리안헤어 도봉한신점 크롤링을 진행합니다.

리안헤어 도봉한신점의 명시적 총 블로그 리뷰 수: 18 

▶▶ 리안헤어 도봉한신점 의 블로그 탭: /hairshop/1222327123/review/ugc 

263963767 헤어살롱나루 양재역점 크롤링을 진행합니다.

헤어살롱나루 양재역점의 명시적 총 블로그 리뷰 수: 318 

▶▶ 헤어살롱나루 양재역점 의 블로그 탭: /hairshop/263963767/review/ugc 

37236557 오다헤어 중앙대점 크롤링을 진행합니다.

오다헤어 중앙대점의 명시적 총 블로그 리뷰 수: 61 

▶▶ 오다헤어 중앙대점 의 블로그 탭: /hairshop/37236557/review/ugc 

해당 엘리먼트를 찾지 못했습니다.


C:\Users\82106\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: `item` has been deprecated and will be removed in a future version
  



★ 박호준헤어 일원점의 블로그 리뷰 썸네일 크롤링 성공!
해당 엘리먼트를 찾지 못했습니다.
1590537785 라리엔 가로수길점 크롤링을 진행합니다.

라리엔 가로수길점의 명시적 총 블로그 리뷰 수: 1042 

▶▶ 라리엔 가로수길점 의 블로그 탭: /hairshop/1590537785/review/ugc 




★ 리안헤어 도봉한신점의 블로그 리뷰 썸네일 크롤링 성공!
해당 엘리먼트를 찾지 못했습니다.



★ 헤어살롱나루 양재역점의 블로그 리뷰 썸네일 크롤링 성공!
해당 엘리먼트를 찾지 못했습니다.


...


★ 오랑헤어 압구정본점의 블로그 리뷰 썸네일 크롤링 성공!
483 ~ 487의 블로그 리뷰 크롤링을 시작합니다.

36772322 리안헤어 이수이마트점 크롤링을 진행합니다.

리안헤어 이수이마트점의 명시적 총 블로그 리뷰 수: 11 

▶▶ 리안헤어 이수이마트점 의 블로그 탭: /hairshop/36772322/review/ugc 

1234677764 박승철헤어스투디오 개포동역점 크롤링을 진행합니다.

박승철헤어스투디오 개포동역점의 명시적 총 블로그 리뷰 수: 233 

▶▶ 박승철헤어스투디오 개포동역점 의 블로그 탭: /hairshop/1234677764/review/ugc 

37346275 이지헤어 연남점 크롤링을 진행합니다.

이지헤어 연남점의 명시적 총 블로그 리뷰 수: 756 

▶▶ 이지헤어 연남점 의 블로그 탭: /hairshop/37346275/review/ugc 

해당 엘리먼트를 찾지 못했습니다.
34635838 이철헤어커커 여의도역점 크롤링을 진행합니다.

이철헤어커커 여의도역점의 명시적 총 블로그 리뷰 수: 141 

▶▶ 이철헤어커커 여의도역점 의 블로그 탭: /hairshop/34635838/review/ugc 

37886017 까리노헤어 크롤링을 진행합니다.

까리노헤어의 명시적 총 블로그 리뷰 수: 42 

▶▶ 까리노헤어 의 블로그 탭: /hairshop/37886017/review/ugc 




★ 리안헤어 이수이마트점의 블로그 리뷰 썸네일 크롤링 성공!
해당 엘리먼트를 찾지 못했습니다.해당 엘리먼트를 찾지 못했습니다.해당 엘리먼트를 찾지 못했습니다.





★ 까리노헤어의 블로그 리뷰 썸네일 크롤링 성공!

★ 박승철헤어스투디오 개포동역점의 블로그 리뷰 썸네일 크롤링 성공!

★ 이지헤어 연남점의 블로그 리뷰 썸네일 크롤링 성공!
해당 엘리먼트를 찾지 못했습니다.



★ 이철헤어커커 여의도역점의 블로그 리뷰 썸네일 크롤링 성공!
488 ~ 492의 블로그 리뷰 크롤링을 시작합니다.

32359012 프리티살롱 청담점 크롤링을 진행합니다.

프리티살롱 청담점의 명시적 총 블로그 리뷰 수: 174 

▶▶ 프리티살롱 청담점 의 블로그 탭: /hairshop/32359012/review/ugc 

1039364681 파란헤어 약수점 크롤링을 진행합니다.

파란헤어 약수점의 명시적 총 블로그 리뷰 수: 189 

▶▶ 파란헤어 약수점 의 블로그 탭: /hairshop/1039364681/review/ugc 

20721082 이철헤어커커 현대백화점 디큐브시티 크롤링을 진행합니다.

이철헤어커커 현대백화점 디큐브시티의 명시적 총 블로그 리뷰 수: 9 

▶▶ 이철헤어커커 현대백화점 디큐브시티 의 블로그 탭: /hairshop/20721082/review/ugc 

1822958774 32비긴 크롤링을 진행합니다.

32비긴의 명시적 총 블로그 리뷰 수: 312 

▶▶ 32비긴 의 블로그 탭: /hairshop/1822958774/review/ugc 

블로그 리뷰가 없습니다!
다음 업소의 크롤링을 진행합니다.
★ 32비긴의 블로그 리뷰 썸네일 크롤링 성공!
★ 아이벨르팜므 발산역점의 블로그 리뷰 썸네일 크롤링 성공!


KeyboardInterrupt: 